# 1. All-or-nothing Thinking

## 1. 모델 정의

### 1-1. 기본 준비

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data1 = pd.read_csv('d01_preprocessed_revised.csv')
data2 = pd.read_csv('d02_preprocessed.csv')
data3 = pd.read_csv('d03_preprocessed.csv')

### 1-2. 데이터 증강 및 구조화

In [ ]:
for idx, i in enumerate(data2['thought']):
    if type(i) != str:
        data2['thought'][idx] = ''

In [ ]:
# has_distortion == 1 필터링

data1 = data1[data1['has_distortion'] == 1].reset_index(drop=True)
data2 = data2[data2['has_distortion'] == 1].reset_index(drop=True)
data3 = data3[data3['has_distortion'] == 1].reset_index(drop=True)

In [ ]:
data1_1 = data1['situation']+' '+data1['thought']
data2_1 = data2['situation']
data3_1 = data3['situation']+' '+data3['thought']

In [ ]:
data1_1.drop_duplicates(inplace = True)
data2_1.drop_duplicates(inplace = True)
data3_1.drop_duplicates(inplace = True)

In [ ]:
def normalize_text(s):
    # Removing articles and punctuation, and standardizing whitespace
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)
bert_config = BertConfig.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Embedding

def tokenize_and_pad(data, tokenizer, max_len=512):
    tokenized_data = []
    for text in data:
        encoded = tokenizer(normalize_text(text), return_tensors="pt", padding='max_length', truncation=True, max_length=max_len)
        tokenized_data.append(encoded)
    return tokenized_data

data1_1_encoded = tokenize_and_pad(data1_1, tokenizer)
data2_1_encoded = tokenize_and_pad(data2_1, tokenizer)
data3_1_encoded = tokenize_and_pad(data3_1, tokenizer)

In [ ]:
data1.columns

Index(['situation', 'thought', 'reframe', 'has_distortion',
       'all-or-nothing thinking', 'comparing and despairing',
       'disqualifying the positive', 'emotional reasoning', 'fortune telling',
       'labeling', 'magnification', 'mind reading', 'overgeneralizing',
       'should statements', 'mental filter', 'personalization and blaming'],
      dtype='object')

In [ ]:
# Add labels
data1_1_labels = list(data1['all-or-nothing thinking'][data1_1.index])
data2_1_labels = list(data2['all-or-nothing thinking'][data2_1.index])
data3_1_labels = list(data3['all-or-nothing thinking'][data3_1.index])

In [ ]:
# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

In [ ]:
class CustomDatasetWithLabels(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {"input_ids": self.data[idx]['input_ids'].squeeze(),
                "attention_mask": self.data[idx]['attention_mask'].squeeze(),
                "y": self.labels[idx]}

In [ ]:
dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=0)

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

### 1-3. 모델 평가 함수 정의

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, dataloader, device="cpu"):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            y = batch["y"].to(device)

            # Get embeddings from the BERT model
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

            logits = model(embeddings)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y.cpu().tolist())

    acc = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return {"accuracy": acc, "f1_macro": f1_macro}

In [ ]:
# Classifier that uses label embeddings to make predictions
class InnerProductClassifier(nn.Module):
    def __init__(self, input_dim, label_embeddings, trainable_label_emb=True):
        super().__init__()
        # Project input features into the same dimension as label embeddings
        self.proj = nn.Linear(input_dim, label_embeddings.size(1))

        if trainable_label_emb:
            # Label embeddings are trainable parameters
            self.label_emb = nn.Parameter(label_embeddings.clone())
        else:
            # Label embeddings are fixed (not updated during training)
            self.register_buffer("label_emb", label_embeddings.clone())

    def forward(self, x):
        # Project input feature vectors
        x_proj = self.proj(x)
        # Compute logits as similarity with each label embedding
        logits = torch.matmul(x_proj, self.label_emb.T)
        return logits

### 1-4. 모델 생성

In [ ]:
# Instantiate the InnerProductClassifier model
model1 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model1.parameters(), lr=2e-4)

## 2. 모델 학습

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model1.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model1(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model1, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/58 [00:02<02:17,  2.41s/it]

Loss: 2.830264091491699


Epoch 1:   3%|▎         | 2/58 [00:04<01:49,  1.96s/it]

Loss: 1.9927647113800049


Epoch 1:   5%|▌         | 3/58 [00:05<01:41,  1.84s/it]

...

Epoch 7:  98%|█████████▊| 57/58 [01:50<00:01,  1.94s/it]

Loss: 0.2854057848453522


Epoch 7: 100%|██████████| 58/58 [01:51<00:00,  1.92s/it]

Loss: 0.000992117915302515


Epoch 7 Validation Accuracy: 0.8522167487684729, F1-macro: 0.721765350877193


## 3. 모델 테스트

In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model1, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8725490196078431, F1-macro: 0.7225360954174513


In [ ]:
distortion_types = [
    'all-or-nothing thinking', 'comparing and despairing',
    'disqualifying the positive', 'emotional reasoning', 'fortune telling',
    'labeling', 'magnification', 'mind reading', 'overgeneralizing',
    'should statements', 'mental filter', 'personalization and blaming'
]

In [ ]:
results_df = pd.DataFrame({
    "distortion_type": distortion_types,
    "test_accuracy": [np.nan] * 12,
    "f1_macro": [np.nan] * 12
})

In [ ]:
current_type = 'all-or-nothing thinking'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 2. Comparing and Despairing

In [ ]:
# Add labels
data1_1_labels = list(data1['comparing and despairing'][data1_1.index])
# comparing and despairing은 data1에만 있음.

# Merging Data
data_encoded = data1_1_encoded
data_labels = data1_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model2 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model2.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model2.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model2(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model2, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:  12%|█▎        | 1/8 [00:01<00:13,  1.91s/it]

Loss: 3.1400747299194336


Epoch 1:  25%|██▌       | 2/8 [00:03<00:11,  1.93s/it]

Loss: 0.5828450918197632


Epoch 1:  38%|███▊      | 3/8 [00:05<00:09,  1.93s/it]

...

Epoch 7:  88%|████████▊ | 7/8 [00:13<00:01,  1.93s/it]

Loss: 0.0


Epoch 7: 100%|██████████| 8/8 [00:13<00:00,  1.74s/it]

Loss: 1.5492181777954102


Epoch 7 Validation Accuracy: 1.0, F1-macro: 1.0


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model2, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.92, F1-macro: 0.4791666666666667


In [ ]:
current_type = 'comparing and despairing'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 3. Disqualifying the Positive

In [ ]:
# Add labels
data1_1_labels = list(data1['disqualifying the positive'][data1_1.index])
data2_1_labels = list(data2['disqualifiying the positive'][data2_1.index]) # 자료에서 오타 있었음!
data3_1_labels = list(data3['disqualifying the positive'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model3 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model3.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model3.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model3(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model3, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/63 [00:01<01:58,  1.91s/it]

Loss: 2.279573917388916


Epoch 1:   3%|▎         | 2/63 [00:03<01:57,  1.93s/it]

Loss: 1.9934914112091064


Epoch 1:   5%|▍         | 3/63 [00:05<01:55,  1.93s/it]

...

Epoch 7:  98%|█████████▊| 62/63 [02:00<00:01,  1.95s/it]

Loss: 0.0


Epoch 7: 100%|██████████| 63/63 [02:01<00:00,  1.93s/it]

Loss: 0.06817980110645294


Epoch 7 Validation Accuracy: 0.9901477832512315, F1-macro: 0.4975247524752475


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model3, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.9901960784313726, F1-macro: 0.4975369458128079


In [ ]:
current_type = 'disqualifying the positive'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 4. Emotional Reasoning

In [ ]:
# Add labels
data1_1_labels = list(data1['emotional reasoning'][data1_1.index])
data2_1_labels = list(data2['emotional reasoning'][data2_1.index])
data3_1_labels = list(data3['emotional reasoning'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model4 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model4.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model4.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model4(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model4, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/58 [00:01<01:49,  1.92s/it]

Loss: 8.413392066955566


Epoch 1:   3%|▎         | 2/58 [00:03<01:48,  1.93s/it]

Loss: 3.1930437088012695


Epoch 1:   5%|▌         | 3/58 [00:05<01:46,  1.94s/it]

...

Epoch 7:  97%|█████████▋| 56/58 [01:48<00:03,  1.94s/it]

Loss: 0.18926510214805603


Epoch 7: 100%|██████████| 58/58 [01:50<00:00,  1.91s/it]

Loss: 0.06566423177719116
Loss: 7.544970139861107e-05


Epoch 7 Validation Accuracy: 0.8866995073891626, F1-macro: 0.6880053458068828


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model4, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8872549019607843, F1-macro: 0.6739628934750885


In [ ]:
current_type = 'emotional reasoning'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 5. Fortune Telling

In [ ]:
# Add labels
data1_1_labels = list(data1['fortune telling'][data1_1.index])
data2_1_labels = list(data2['fortune telling'][data2_1.index])
data3_1_labels = list(data3['fortune telling'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model5 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model5.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model5.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model5(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model5, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/55 [00:01<01:43,  1.92s/it]

Loss: 9.603532791137695


Epoch 1:   4%|▎         | 2/55 [00:03<01:42,  1.93s/it]

Loss: 2.1768786907196045


Epoch 1:   5%|▌         | 3/55 [00:05<01:40,  1.94s/it]

...

Epoch 7:  98%|█████████▊| 54/55 [01:45<00:01,  1.95s/it]

Loss: 0.8189567923545837


Epoch 7: 100%|██████████| 55/55 [01:45<00:00,  1.92s/it]

Loss: 0.00138014554977417


Epoch 7 Validation Accuracy: 0.8719211822660099, F1-macro: 0.7307142857142856


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model5, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8823529411764706, F1-macro: 0.7166666666666667


In [ ]:
current_type = 'fortune telling'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 6. Labeling

In [ ]:
# Add labels
data1_1_labels = list(data1['labeling'][data1_1.index])
data2_1_labels = list(data2['labeling'][data2_1.index])
data3_1_labels = list(data3['labeling'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model6 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model6.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model6.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model6(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model6, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/54 [00:01<01:41,  1.92s/it]

Loss: 4.968138694763184


Epoch 1:   4%|▎         | 2/54 [00:03<01:40,  1.93s/it]

Loss: 2.6190247535705566


Epoch 1:   6%|▌         | 3/54 [00:05<01:38,  1.94s/it]

...

Epoch 7:  98%|█████████▊| 53/54 [01:43<00:01,  1.95s/it]

Loss: 0.6914868354797363


Epoch 7: 100%|██████████| 54/54 [01:44<00:00,  1.93s/it]

Loss: 0.09898076206445694


Epoch 7 Validation Accuracy: 0.8177339901477833, F1-macro: 0.7198328919392741


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model6, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.7990196078431373, F1-macro: 0.6781962987187873


In [ ]:
current_type = 'labeling'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 7. Magnification

In [ ]:
# Add labels
data1_1_labels = list(data1['labeling'][data1_1.index])
data2_1_labels = list(data2['labeling'][data2_1.index])
data3_1_labels = list(data3['labeling'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model7 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model7.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model7.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model7(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model7, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/54 [00:01<01:42,  1.93s/it]

Loss: 2.9581737518310547


Epoch 1:   4%|▎         | 2/54 [00:03<01:40,  1.94s/it]

Loss: 2.8381199836730957


Epoch 1:   6%|▌         | 3/54 [00:05<01:39,  1.95s/it]

...

Epoch 7:  98%|█████████▊| 53/54 [01:43<00:01,  1.94s/it]

Loss: 1.0412416458129883


Epoch 7: 100%|██████████| 54/54 [01:44<00:00,  1.93s/it]

Loss: 0.77287757396698


Epoch 7 Validation Accuracy: 0.8571428571428571, F1-macro: 0.6228457940931513


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model7, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8431372549019608, F1-macro: 0.6353072625698324


In [ ]:
current_type = 'magnification'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 8. Mind Reading

In [ ]:
# Add labels
data1_1_labels = list(data1['mind reading'][data1_1.index])
data2_1_labels = list(data2['mind reading'][data2_1.index])
data3_1_labels = list(data3['mind reading'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model8 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model8.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model8.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model8(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model8, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/52 [00:01<01:39,  1.95s/it]

Loss: 13.617969512939453


Epoch 1:   4%|▍         | 2/52 [00:03<01:38,  1.98s/it]

Loss: 1.2222529649734497


Epoch 1:   6%|▌         | 3/52 [00:05<01:37,  2.00s/it]

...

Epoch 7:  98%|█████████▊| 51/52 [01:39<00:01,  1.94s/it]

Loss: 0.508968710899353


Epoch 7: 100%|██████████| 52/52 [01:39<00:00,  1.92s/it]

Loss: 1.1984925270080566


Epoch 7 Validation Accuracy: 0.8768472906403941, F1-macro: 0.71969069317868


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model8, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8431372549019608, F1-macro: 0.6473638720829732


In [ ]:
current_type = 'mind reading'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 9. Overgeneralizing

In [ ]:
# Add labels
data1_1_labels = list(data1['overgeneralizing'][data1_1.index]) # data1에서는 명칭이 다름.
data2_1_labels = list(data2['overgeneralization'][data2_1.index])
data3_1_labels = list(data3['overgeneralization'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model9 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model9.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model9.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model9(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model9, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/53 [00:01<01:39,  1.91s/it]

Loss: 3.3430519104003906


Epoch 1:   4%|▍         | 2/53 [00:03<01:38,  1.92s/it]

Loss: 6.321318626403809


Epoch 1:   6%|▌         | 3/53 [00:05<01:36,  1.93s/it]

...

Epoch 7:  98%|█████████▊| 52/53 [01:41<00:01,  1.95s/it]

Loss: 0.1818055659532547


Epoch 7: 100%|██████████| 53/53 [01:42<00:00,  1.94s/it]

Loss: 0.25177276134490967


Epoch 7 Validation Accuracy: 0.8669950738916257, F1-macro: 0.5778975741239892


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model9, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.7941176470588235, F1-macro: 0.48546721114580826


In [ ]:
current_type = 'overgeneralizing'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 10. Should Statements

In [ ]:
# Add labels
data1_1_labels = list(data1['should statements'][data1_1.index])
data2_1_labels = list(data2['should statements'][data2_1.index])
data3_1_labels = list(data3['should statements'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model10 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model10.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model10.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model10(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model10, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/59 [00:01<01:52,  1.93s/it]

Loss: 15.077157974243164


Epoch 1:   3%|▎         | 2/59 [00:03<01:50,  1.94s/it]

Loss: 1.3542100191116333


Epoch 1:   5%|▌         | 3/59 [00:05<01:49,  1.95s/it]

...

Epoch 7:  98%|█████████▊| 58/59 [01:52<00:01,  1.95s/it]

Loss: 0.19606003165245056


Epoch 7: 100%|██████████| 59/59 [01:53<00:00,  1.92s/it]

Loss: 0.011419190093874931


Epoch 7 Validation Accuracy: 0.9261083743842364, F1-macro: 0.5858833129334966


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model10, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.9313725490196079, F1-macro: 0.6319587628865979


In [ ]:
current_type = 'should statements'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 11. Mental Filter

In [ ]:
# Add labels
data1_1_labels = list(data1['mental filter'][data1_1.index])
data2_1_labels = list(data2['mental filter'][data2_1.index])
data3_1_labels = list(data3['mental filter'][data3_1.index])

# Merging Data
data_encoded = data1_1_encoded + data2_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data2_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model11 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model11.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model11.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model11(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model11, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   2%|▏         | 1/59 [00:01<01:50,  1.91s/it]

Loss: 3.844313859939575


Epoch 1:   3%|▎         | 2/59 [00:03<01:49,  1.93s/it]

Loss: 1.3824689388275146


Epoch 1:   5%|▌         | 3/59 [00:05<01:48,  1.94s/it]

...

Epoch 7:  97%|█████████▋| 57/59 [01:50<00:03,  1.94s/it]

Loss: 0.26722967624664307


Epoch 7: 100%|██████████| 59/59 [01:52<00:00,  1.91s/it]

Loss: 0.3195909261703491
Loss: 6.556505240951083e-07


Epoch 7 Validation Accuracy: 0.9113300492610837, F1-macro: 0.47680412371134023


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model11, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.9313725490196079, F1-macro: 0.5446428571428572


In [ ]:
current_type = 'mental filter'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# 12. Personalization and Blaming

In [ ]:
# Add labels
data1_1_labels = list(data1['personalization and blaming'][data1_1.index])
data3_1_labels = list(data3['personalization and blaming'][data3_1.index]) # data2에는 없음.

# Merging Data
data_encoded = data1_1_encoded + data3_1_encoded
data_labels = data1_1_labels + data3_1_labels

dataset_with_labels = CustomDatasetWithLabels(data_encoded, data_labels)

# Define proportions for splitting
train_size = int(0.8 * len(dataset_with_labels))
val_size = int(0.1 * len(dataset_with_labels))
test_size = len(dataset_with_labels) - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset_with_labels, [train_size, val_size, test_size])

In [ ]:
dataset_encoded = dataset_with_labels.data
dataset_labels = dataset_with_labels.labels

# Convert list of BatchEncoding objects to a 2D numpy array for SMOTE features
feature_vectors = []
max_len = 512 # Assuming this from tokenize_and_pad function
for encoded_dict in dataset_encoded:
    input_ids_np = encoded_dict['input_ids'].squeeze().cpu().numpy()
    attention_mask_np = encoded_dict['attention_mask'].squeeze().cpu().numpy()
    # Concatenate input_ids and attention_mask to create a single feature vector per sample
    feature_vectors.append(np.concatenate([input_ids_np, attention_mask_np]))

# 'temp1' becomes the feature matrix, 'temp2' becomes the labels vector
temp1 = np.array(feature_vectors)
temp2 = np.array(dataset_labels)

# Apply SMOTE
temp1_resampled, temp2_resampled = smote.fit_resample(temp1, temp2)

# Reconstruct data_encoded and data_labels from the resampled data
reconstructed_data_encoded = []
for resampled_features in temp1_resampled:
    # Split the combined feature vector back into input_ids and attention_mask parts
    reconstructed_input_ids = resampled_features[:max_len]
    reconstructed_attention_mask = resampled_features[max_len:]

    # Convert back to torch tensors and a dictionary format compatible with CustomDatasetWithLabels
    input_ids_tensor = torch.tensor(reconstructed_input_ids, dtype=torch.long).unsqueeze(0)
    attention_mask_tensor = torch.tensor(reconstructed_attention_mask, dtype=torch.long).unsqueeze(0)

    reconstructed_data_encoded.append({
        'input_ids': input_ids_tensor,
        'attention_mask': attention_mask_tensor
    })

# Update data_encoded and data_labels with the resampled data
dataset_encoded = reconstructed_data_encoded
dataset_labels = temp2_resampled.tolist()

train_dataset = CustomDatasetWithLabels(dataset_encoded, dataset_labels)

In [ ]:
# Create DataLoaders for each set
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

input_dim = bert_config.hidden_size
num_labels = len(set(data_labels))
label_emb = torch.randn(num_labels, input_dim)

In [ ]:
# Instantiate the InnerProductClassifier model
model12 = InnerProductClassifier(input_dim, label_emb).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.AdamW(model12.parameters(), lr=2e-4)

In [ ]:
EPOCHS = 7

# Move the BERT model to the device
bert_model.to(device)

for epoch in range(EPOCHS):
    model12.train()
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        y = batch["y"].to(device)

        # Get embeddings from the BERT model
        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state[:, 0, :] # Get the [CLS] token embedding

        # Pass embeddings to the InnerProductClassifier
        logits = model12(embeddings)
        loss = criterion(logits, y)

        # Backpropagate and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

    # Evaluate on the validation set after each epoch
    val_metrics = evaluate(model12, val_dataloader, device)
    print(f"Epoch {epoch+1} Validation Accuracy: {val_metrics['accuracy']}, F1-macro: {val_metrics['f1_macro']}")

Epoch 1:   9%|▉         | 1/11 [00:01<00:19,  1.91s/it]

Loss: 5.8179216384887695


Epoch 1:  18%|█▊        | 2/11 [00:03<00:17,  1.93s/it]

Loss: 1.3913652896881104


Epoch 1:  27%|██▋       | 3/11 [00:05<00:15,  1.94s/it]

...

Epoch 7:  91%|█████████ | 10/11 [00:19<00:01,  1.94s/it]

Loss: 0.3498877286911011


Epoch 7: 100%|██████████| 11/11 [00:21<00:00,  1.91s/it]

Loss: 0.34534475207328796


Epoch 7 Validation Accuracy: 0.8837209302325582, F1-macro: 0.7734457323498419


In [ ]:
# Evaluate on the test set
test_metrics = evaluate(model12, test_dataloader, device)
print(f"Test Accuracy: {test_metrics['accuracy']}, F1-macro: {test_metrics['f1_macro']}")

Test Accuracy: 0.8888888888888888, F1-macro: 0.8


In [ ]:
current_type = 'personalization and blaming'

results_df.loc[
    results_df["distortion_type"] == current_type,
    ["test_accuracy", "f1_macro"]
] = [test_metrics['accuracy'], test_metrics['f1_macro']]

# Accuracy

In [ ]:
results_df

,distortion_type,test_accuracy,f1_macro
0,all-or-nothing thinking,0.872549,0.722536
1,comparing and despairing,0.920000,0.479167
2,disqualifying the positive,0.990196,0.497537
3,emotional reasoning,0.887255,0.673963
4,fortune telling,0.882353,0.716667
5,labeling,0.799020,0.678196
6,magnification,0.843137,0.635307
7,mind reading,0.843137,0.647364
8,overgeneralizing,0.794118,0.485467
9,should statements,0.931373,0.631959


# Savings

In [ ]:
import torch
import os

output_dir = "./saved_smote_model"
os.makedirs(output_dir, exist_ok=True)

# Save the entire Binary Classifier model's state_dict
torch.save(model1.state_dict(), os.path.join(output_dir, "model1_weights.pth"))
torch.save(model2.state_dict(), os.path.join(output_dir, "model2_weights.pth"))
torch.save(model3.state_dict(), os.path.join(output_dir, "model3_weights.pth"))
torch.save(model4.state_dict(), os.path.join(output_dir, "model4_weights.pth"))
torch.save(model5.state_dict(), os.path.join(output_dir, "model5_weights.pth"))
torch.save(model6.state_dict(), os.path.join(output_dir, "model6_weights.pth"))
torch.save(model7.state_dict(), os.path.join(output_dir, "model7_weights.pth"))
torch.save(model8.state_dict(), os.path.join(output_dir, "model8_weights.pth"))
torch.save(model9.state_dict(), os.path.join(output_dir, "model9_weights.pth"))
torch.save(model10.state_dict(), os.path.join(output_dir, "model10_weights.pth"))
torch.save(model11.state_dict(), os.path.join(output_dir, "model11_weights.pth"))
torch.save(model12.state_dict(), os.path.join(output_dir, "model12_weights.pth"))

print(f"Model saved to {output_dir}")

Model saved to ./saved_smote_model
